In [22]:
## 通过密码生成验证码
import hashlib
password = "XXXXXXX";
m=hashlib.md5()
b=password.encode(encoding='UTF-8')
m.update(b)
import base64
b64 = base64.b64encode(m.digest())
verifyCode = bytes.decode(b64)
print(verifyCode)
## 生成验证密码完成

xZBe7RkIVMwlQ6FssvY1tg==


In [23]:
##通过验证密码取得token
#windUser:用户名，verifyCode：账户密码加密后生成的字符串
#请务必注意：不要每次调用接口都重新取得token，太多调用会被认为账户异常，最终影响wft账户的正常使用
import requests
#token取得
#请务必注意：不要每次调用接口都重新取得token，太多调用会被认为账户异常，最终影响wft账户的正常使用
tokenResponse = requests.get("http://eapi.wind.com.cn/wind.ent.risk/openapi/getToken?windUser=18702198017&verifyCode="+requests.utils.quote(verifyCode))
import json
jsonToken = json.loads(tokenResponse.text)
token = ""
if jsonToken["errorCode"] == 0:
    token = jsonToken["source"]["token"]
    pass
else : print(str(jsonToken["errorCode"])+":"+jsonToken["message"])
##token取得完成 (运行一次就行)
token 

'b4380d99-3b83-4c8e-991c-0b31a90dc6cb'

In [24]:
##通过token取查找企业的windId，然后查询企业信息
def getInfoFromWind(corpName):
    windId = ""
    searchResult = requests.get("http://eapi.wind.com.cn/wind.ent.risk/openapi/searchcorplist?corpName="+corpName+"&token="+token)
    #如果errorCode = 403时，则说明token过期，或错误，需要通过上面接口重新取得token
    searchResultJson = json.loads(searchResult.text)
    if searchResultJson["errorCode"] == 0:
        if searchResultJson.get("source") is not None and searchResultJson.get("source")["total"]>0:
            windId = searchResultJson["source"]["items"][0]["windId"]
        pass
    else : 
        pass #print(str(searchResultJson["errorCode"])+":"+searchResultJson["message"])
    ##查找企业id完成

    #定义端口    
    port = 'A002'
    
    #查询企业信息
    url="http://eapi.wind.com.cn/wind.ent.risk/openapi/corpinfo/"+port+"?windId="+windId+"&token="+token
    Result= requests.get(url)
    ResultJson = json.loads(Result.text)
    #print (ResultJson)
    if ResultJson["errorCode"] == 0:
        data_res = ResultJson["source"]
        #status = '!!ok'
        #print(data_res)
        pass
    else : 
        data_res = {'corpName': corpName, 'hisName': '', 'creditId': ' ', 'orgCode': '', 'regNum': '', 'legalRepName': '', 'regCap': '', 'regCapCur': '', 'establishDate': '', 'openFrom': '', 'openTo': '', 'controllerName': '', 'corpType': '', 'regStatus': '', 'businessScope': '', 'regOrgCode': '', 'cancelDate': '', 'revokeDate': '', 'accCap': '', 'accCapCur': '', 'regRegion': '', 'regProvince': '', 'regCity': '', 'regOrg': '', 'description': '', 'regAddress': '', 'email': '', 'telNum': '', 'website': '', 'industryCode': '', 'industryName': '', 'windIndustry': '', 'windIndustryTheme': ''}   
        print(str(ResultJson["errorCode"])+":"+ ResultJson["message"]+":"+ ResultJson["source"])
        #status = ResultJson["Source"]
    ##查询相关数据完成
    return data_res

In [30]:
import time
import pandas as pd
df = pd.read_excel(r'/Users/HongX/Desktop/汇总台账.xlsx')
corpNameList = df[r'发行人全称'].values.tolist()
corpNameList = corpNameList[0:10]
#print (corpNameList)

df_results = pd.DataFrame()

count = 0
for corp in corpNameList:
    count +=1
    result_tmp = getInfoFromWind(corp)
    df_tmp = pd.DataFrame.from_dict(result_tmp, orient='index').T
    frames = [df_results, df_tmp]
    df_results = pd.concat(frames)
    df_results.to_csv('/Users/HongX/Desktop/corp_info_from_Wind.csv', encoding='utf_8_sig')
    print (count, '/', len(corpNameList), corp)
    time.sleep(3)
    
df_results.to_csv('/Users/HongX/Desktop/corp_info_from_Wind.csv', encoding='utf_8_sig')
df_results.head()

1 / 10 东营银行股份有限公司
2 / 10 辽阳银行股份有限公司
3 / 10 桂林银行股份有限公司
4 / 10 湖州银行股份有限公司
5 / 10 浙江德清农村商业银行股份有限公司
6 / 10 东莞农村商业银行股份有限公司
7 / 10 日照银行股份有限公司
8 / 10 江苏金融租赁股份有限公司
9 / 10 浙江苍南农村商业银行股份有限公司
10 / 10 西藏金融租赁有限公司


,corpName,hisName,creditId,orgCode,regNum,legalRepName,regCap,regCapCur,establishDate,openFrom,...,regOrg,description,regAddress,email,telNum,website,industryCode,industryName,windIndustry,windIndustryTheme
0,东营银行股份有限公司,东营市商业银行股份有限公司,91370000732595249Q,732595249,370000018084402,高传星,273782.8297,人民币,20050822,2005-08-22,...,山东省市场监督管理局,东营银行股份有限公司于2005年08月22日在山东省工商行政管理局登记成立。法定代表人高传星...,山东省东营市东营区府前大街87-1号,boardoffice@dyccb.net,0546-8150823,"http://www.dyccb.net,http://www.dyccb.net/",J6620,货币银行服务,区域性银行,银行
0,辽阳银行股份有限公司,辽阳市商业银行股份有限公司,912110004645009004,464500900,211000004009417,徐雅清,263900.5242,人民币,19970325,1997-03-25,...,辽阳市市场监督管理局,辽阳银行是一家由国有股、企业法人股、自然人股组成的具有独立法人资格的股份制商业银行，在本地和...,辽阳市民主路２１号,liaoyangcb@163.com,0419-3733052,www.bankofliaoyang.net/www.bankofliaoyang.cn/w...,J6620,货币银行服务,区域性银行,银行
0,桂林银行股份有限公司,桂林市商业银行股份有限公司,91450300198879064G,198879064,450300000003307,吴东,500000.0,,19970328,1997-03-28,...,桂林市市场监督管理局,桂林银行成立于1997年3月，是由桂林市财政、地方优势企业和城镇居民等众多股东共同发起投立的...,桂林市临桂区公园北路8号,sd@guilinbankcom.cn,0773-3851609,"http://www.guilibank.com.cn,http://www.guilinb...",J6620,货币银行服务,区域性银行,银行
0,湖州银行股份有限公司,湖州市商业银行股份有限公司,91330500717611212U,717611212,330500000014512,吴继平,101337.2667,人民币,19980617,1998-06-17,...,湖州市市场监督管理局,湖州银行是中国浙江省湖州市的一家区域性股份制商业银行。前身是1998年6月17日成立的湖州市...,浙江省湖州市红旗路268号,,2890021,http://www.hzccb.net,J66,货币金融服务,区域性银行,银行
0,浙江德清农村商业银行股份有限公司,浙江德清农村合作银行,9133050076962455XQ,76962455X,330521000017820,陈春仿,80850.0005,人民币,20080821,2008-08-21,...,湖州市市场监督管理局,浙江德清农村合作银行是全国农村信用社第二轮改革中，经中国银行业监督管理委员会批准设立的股份合...,浙江省德清县武康街道沈长圩街50号,,0572-8065931,http://www.deqingbank.com,J6620,货币银行服务,区域性银行,银行


In [ ]:
#ENDING

In [34]:
##TEST Only
company = '无锡市新发集团有限公司'
result_dict = getInfoFromWind(company)
result_dict

{'corpName': '无锡市新发集团有限公司',
 'hisName': '无锡市开发区发展总公司,无锡市新区发展集团有限公司,无锡市新区经济发展集团总公司',
 'creditId': '9132021325045518X3',
 'orgCode': '25045518X',
 'regNum': '320213000001584',
 'legalRepName': '黄际洲',
 'regCap': '1202869.951863',
 'regCapCur': '人民币',
 'establishDate': '19920721',
 'openFrom': '1992-07-21',
 'openTo': '',
 'controllerName': '无锡市人民政府',
 'corpType': '有限责任公司(国有独资)',
 'regStatus': '存续（在营、开业、在册）',
 'businessScope': '股权投资、国有土地开发、基础设施建设、公用设施开发经营、城市建设综合开发、自有房产经营及物业管理、企业管理咨询、投资咨询、园艺经营（含苗木销售）。（依法须经批准的项目，经相关部门批准后方可开展经营活动）',
 'regOrgCode': '320214',
 'cancelDate': '',
 'revokeDate': '',
 'accCap': '',
 'accCapCur': '',
 'regRegion': '无锡市新吴区汇融广场2－402、502、602、702、801、901',
 'regProvince': '江苏省',
 'regCity': '无锡市',
 'regOrg': '无锡国家高新技术产业开发区（无锡市新吴区）行政审批局',
 'description': '无锡市新发集团有限公司（下称：新发集团）是无锡高新区（新吴区）直属国有独资公司，作为区域开发的组织和实施主体，长期承担政府性的开发建设、融资及对外投资任务；承担区域科技载体、公用事业、市政绿化等基础设施的建设和运营；从事高新技术企业投资、国有资产经营和资本运作等。近年来，正在逐步实现从原来单一的保障平台功能向投资决策和运营管理职能转变。目前，新发集团注册资本119亿元，净资产约157亿元，总资产约406亿元。截至2018年末，集团共有

In [ ]:
#
url_check="http://eapi.wind.com.cn/wind.ent.risk/openapi/verifyCorpInfo/A002?pageIndex=1&pageSize=10&windId="+windId+"&keyword=雷军&token="+token

In [19]:
##通过windId取相关类型数据，具体参数参看每个接口的输入参数
#基本信息的url样例：http://eapi.wind.com.cn/wind.ent.risk/openapi/corpinfo/接口id?pageIndex=1&pageSize=10&windId=上述查出的id&token=上述生成的token
#核查类接口url样例：http://eapi.wind.com.cn/wind.ent.risk/openapi/verifyCorpInfo/接口id?pageIndex=1&pageSize=10&windId=上述查出的id&根据接口参数填入参数（keyword=雷军 或者 beneficiary=雷军）&token=上述生成的token
Result= requests.get(url_a)
ResultJson = json.loads(Result.text)
if ResultJson["errorCode"] == 0:
    data = ResultJson["source"]
    print(data)
    pass
else : print(str(ResultJson["errorCode"])+":"+ ResultJson["message"])
##查询相关数据完成

NameError: name 'url_a' is not defined

In [31]:
from DataAPI import *

ModuleNotFoundError: No module named 'DataAPI'

In [33]:
!pip install DataAPI

  Using cached DataAPI-0.8.2.tar.gz (732 kB)
  Using cached Finance-Python-0.8.1.tar.gz (3.1 MB)
    ERROR: Command errored out with exit status 1:
     command: /Users/HongX/anaconda3/envs/wind/bin/python -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/bp/hxj408gn1pl64k326jnyvfzw0000gn/T/pip-install-ox_barwo/Finance-Python/setup.py'"'"'; __file__='"'"'/private/var/folders/bp/hxj408gn1pl64k326jnyvfzw0000gn/T/pip-install-ox_barwo/Finance-Python/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /private/var/folders/bp/hxj408gn1pl64k326jnyvfzw0000gn/T/pip-install-ox_barwo/Finance-Python/pip-egg-info
         cwd: /private/var/folders/bp/hxj408gn1pl64k326jnyvfzw0000gn/T/pip-install-ox_barwo/Finance-Python/
    Complete output (5 lines):
    Traceback (most recent call last):
      File "<string>", line 1, in <module>


In [ ]:
partyFullName primeOperating regProvince regCity opaScope profile